<a href="https://colab.research.google.com/github/wiebkepeters/NI_PJ_AED/blob/dataloading/SED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 2.8MB/s 


In [2]:
import torch
import torchaudio
from torchaudio import functional as F
from torchaudio import transforms as T

In [3]:
torchaudio.set_audio_backend("sox_io")

**( 1. UPLOAD NEEDED FILES (.py, etc.) AND MIMIC FOLDER STRUCTURE OF REPO (EDIT/ADAPT UPLOADS IF NECESSARY) IF FILES NEED TO BE LOADED FROM REPO (NOT DRIVE) )**

In [4]:
# edit if necessary
directories = ['data_feeders', 'experiments', 'models', 'settings', 'tools']

In [7]:
from google.colab import files

# upload directory by directory 
for dir in directories:
  !mkdir dir
  uploaded = files.upload()
  !mv *.* dir

Saving _data_loader_functions.py to _data_loader_functions.py
Saving __init__.py to __init__.py
Saving _tut_sed_synthetic_2016.py to _tut_sed_synthetic_2016.py
mv: target 'dir/' is not a directory


mv: target 'dir/' is not a directory


mv: target 'dir/' is not a directory


mv: target 'dir/' is not a directory


mv: target 'dir/' is not a directory


In [10]:
!ls

data_feeders  experiments  models  sample_data	settings  tools


**2. MOUNT DRIVE TO ACCESS DATA (AND FILES IF NOT LOADED FROM REPO)**

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
# Change directory if needed
%cd drive/MyDrive/Data_Code/

/content/drive/.shortcut-targets-by-id/1nC9alf1XXss0MliDJ7UIwgKnGiUiWn01/Data_Code


**3. CD TO drive/MyDrive/Data_Code/ (%cd drive/MyDrive/Data_Code/) IF FILES FROM DRIVE ARE USED**

In [ ]:
# %cd drive/MyDrive/Data_Code/

**( 4. SET PATHS TO DATA ) not necessary at this point**

In [1]:
# # set paths to data
# train_data = './data/synthetic/training'
# validata_data = './data/synthetic/validate'
# test_data = './data/synthetic/test'

NameError: ignored

**5. CHANGE PARAMS IF NECESSARY (MAKE SURE THAT setting_file CONTAINS CORRECT PATH TO DATA)**

In [7]:
model_to_use = 'baseline'#@param {type:"string"}
config_file = 'synthetic_2016_k_33_d_1_1'#@param {type: "string"}
file_ext = '.yaml'#@param {type: "string"}
settings_dir = './settings'#@param {type:"string"}


**6. START EXPERIMENTS**

In [8]:
from pathlib import Path

from experiments.experiment_process import do_process
from tools.file_io import load_settings_file
from tools.various import get_argument_parser
from tools.printing import InformAboutProcess, date_and_time

In [ ]:
with InformAboutProcess('Loading settings'):
        settings = load_settings_file(
            file_name=Path(f'{config_file}{file_ext}'),
            settings_dir=Path(settings_dir))

do_process(
    settings_path=None,
    settings=settings,
    model_to_use=model_to_use)